# affinity label

In [1]:
from shutil import copyfile
import json
import os, cv2
from pycocotools import mask as decode_mask
import matplotlib.pyplot as plt
import numpy as np

### Train

In [2]:
key_dict = {
'fence':1,
'branch':2,
'log':3,
'hanger':4,
'wire':5,
'antenna':6
} 

categories = ['fence','branch','log','hanger','wire',]

In [4]:
# 아 seg 지우고 다시 해야함..
'''name_count = 0
for category in categories:
    file_names = os.listdir("./"+category+"/train/image/")
    
    try:
        file_names.remove('.ipynb_checkpoints')
    except:
        pass
    
    for name in file_names:
        src_path = "./"+category+"/train/image/"+name
        dst_path = "./_semantic/image/"+str(name_count)+".jpg"
        copyfile(src_path,dst_path)
        
        name = name.split(".")[0]
        src_path = "./"+category+"/train/instance_map/"+name+".png"
        dst_path = "./_semantic/label/"+str(name_count)+".png"
        label_img = cv2.imread(src_path)
        label_img = np.where(label_img>=1,key_dict[category],0)
        cv2.imwrite(dst_path,label_img)
        
        name_count+=1'''

'name_count = 0\nfor category in categories:\n    file_names = os.listdir("./"+category+"/train/image/")\n    \n    try:\n        file_names.remove(\'.ipynb_checkpoints\')\n    except:\n        pass\n    \n    for name in file_names:\n        src_path = "./"+category+"/train/image/"+name\n        dst_path = "./_semantic/image/"+str(name_count)+".jpg"\n        copyfile(src_path,dst_path)\n        \n        name = name.split(".")[0]\n        src_path = "./"+category+"/train/instance_map/"+name+".png"\n        dst_path = "./_semantic/label/"+str(name_count)+".png"\n        label_img = cv2.imread(src_path)\n        label_img = np.where(label_img>=1,key_dict[category],0)\n        cv2.imwrite(dst_path,label_img)\n        \n        name_count+=1'

In [3]:
def translate_image_8_directions(image, d):
    height, width = image.shape[:2]
    directions = [
        (-d, -d),  # 1. x-d, y-d
        (0, -d),   # 2. x, y-d
        (d, -d),   # 3. x+d, y-d
        (-d, 0),   # 4. x-d, y
        (d, 0),    # 5. x+d, y
        (-d, d),   # 6. x-d, y+d
        (0, d),    # 7. x, y+d
        (d, d)     # 8. x+d, y+d
    ]
    
    initial = True
    for (tx, ty) in directions:
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        trans_img = cv2.warpAffine(image, M, (width, height))
        trans_img = np.expand_dims(trans_img,2)
        
        if initial:
            translated_images = trans_img
            initial=False
        else:
            translated_images = np.concatenate([translated_images,trans_img],-1)
    
    return translated_images

In [4]:
name_count = 0
for category in categories:
    image_path = "./"+category+"/train/image/"
    json_path = "./"+category+"/train/coco_format-mask_encoding=rle-instances_2017.json"
    
    with open(json_path,'r') as file:
        json_data = json.load(file)

    data_dict = dict()
    distances = [1,2,4,8,16,32,64]

    for idx,annotation_data in enumerate(json_data['annotations']):  # type(annotation_data) = dict
        key = int(annotation_data['image_id'])
        if key not in data_dict:
            data_dict[key] = set()
        data_dict[key].add(idx)

    for image_id in data_dict:
        # 데이터 매치 문제없는지 확인
        for idx in data_dict[image_id]:
            assert image_id == json_data['annotations'][idx]['image_id'], "key:value idx match error"
        # image id랑 key랑 같은지 확인
        assert image_id == json_data['images'][image_id-1]['id'], "img index:key match error"

        image_name = json_data['images'][image_id-1]['file_name']
        #image = cv2.imread(image_path+image_name)

        #--------intance label create--------
        # 객체 id 초기화
        instance_id = 1
        # 라벨 데이터 틀 생성
        instance_label = np.zeros((1024,1024))
        for i in data_dict[image_id]:
            binary_mask = decode_mask.decode(json_data['annotations'][i]['segmentation'])  # 1 or 0
            binary_mask=np.where(binary_mask==1,instance_id,0)
            instance_label += binary_mask
            instance_id+=1
        #--------intance label complete--------

        #--------affinity label create--------
        initial = True
        for d in distances:  # 총 7개의 거리
            img_8 = translate_image_8_directions(instance_label,d)  # d 거리에 대해 8방향으로 이동한 이미지 생성
            if initial:
                affinity_label = img_8
                initial = False
            else:
                affinity_label = np.concatenate([affinity_label,img_8],-1)

        ins_label_56 = np.repeat(np.expand_dims(instance_label,-1), 56, axis=2)
        affinity_label = np.where((affinity_label == ins_label_56)&(ins_label_56>0), 1, 0)  # instance label과 객체 id가 같으면 1 다르면 0으로
        #--------affinity label complete--------

        # 이미지 및 affinity 라벨 저장
        #src_path = image_path+image_name
        #dst_path = "./_affinity/train/image/"+str(name_count)+".jpg"
        #copyfile(src_path,dst_path)

        dst_path = "./_affinity/train/label/"+str(name_count)+".npz"
        np.savez_compressed(dst_path,data=affinity_label)

        name_count+=1

In [5]:
categories = ['antenna']
for category in categories:
    image_path = "./"+category+"/train/img/image/"
    json_path = "./"+category+"/train/coco_format-mask_encoding=rle-instances_2017.json"
    
    with open(json_path,'r') as file:
        json_data = json.load(file)

    data_dict = dict()
    distances = [1,2,4,8,16,32,64]

    for idx,annotation_data in enumerate(json_data['annotations']):  # type(annotation_data) = dict
        key = int(annotation_data['image_id'])
        if key not in data_dict:
            data_dict[key] = set()
        data_dict[key].add(idx)

    for image_id in data_dict:

        # 데이터 매치 문제없는지 확인
        for idx in data_dict[image_id]:
            assert image_id == json_data['annotations'][idx]['image_id'], "key:value idx match error"
        # image id랑 key랑 같은지 확인
        assert image_id == json_data['images'][image_id-1]['id'], "img index:key match error"

        image_name = json_data['images'][image_id-1]['file_name']
        #image = cv2.imread(image_path+image_name)

        #--------intance label create--------
        # 객체 id 초기화
        instance_id = 1
        # 라벨 데이터 틀 생성
        instance_label = np.zeros((1024,1024))
        for i in data_dict[key]:
            binary_mask = decode_mask.decode(json_data['annotations'][i]['segmentation'])  # 1 or 0
            binary_mask=np.where(binary_mask==1,instance_id,0)
            instance_label += binary_mask
            instance_id+=1
        #--------intance label complete--------

        #--------affinity label create--------
        initial = True
        for d in distances:  # 총 7개의 거리
            img_8 = translate_image_8_directions(instance_label,d)  # d 거리에 대해 8방향으로 이동한 이미지 생성
            if initial:
                affinity_label = img_8
                initial = False
            else:
                affinity_label = np.concatenate([affinity_label,img_8],-1)

        ins_label_56 = np.repeat(np.expand_dims(instance_label,-1), 56, axis=2)
        affinity_label = np.where((affinity_label == ins_label_56)&(ins_label_56>0), 1, 0)  # instance label과 객체 id가 같으면 1 다르면 0으로
        #--------affinity label complete--------
        
        
        # 이미지 및 affinity 라벨 저장
        # src_path = image_path+image_name
        # dst_path = "./_affinity/train/image/"+str(name_count)+".jpg"
        # copyfile(src_path,dst_path)

        dst_path = "./_affinity/train/label/"+str(name_count)+".npz"
        np.savez_compressed(dst_path,data=affinity_label)

        name_count+=1

### Valid

In [2]:
key_dict = {
'fence':1,  # 92.48
'branch':2,  # 90.08
'log':3,  # 97.27
'hanger':4,  # 92.92
'wire':5,  # 96.85
'antenna':6  # 97.20
} 

categories = ['fence','branch','log','hanger','wire',]

def translate_image_8_directions(image, d):
    height, width = image.shape[:2]
    directions = [
        (-d, -d),  # 1. x-d, y-d
        (0, -d),   # 2. x, y-d
        (d, -d),   # 3. x+d, y-d
        (-d, 0),   # 4. x-d, y
        (d, 0),    # 5. x+d, y
        (-d, d),   # 6. x-d, y+d
        (0, d),    # 7. x, y+d
        (d, d)     # 8. x+d, y+d
    ]
    
    initial = True
    for (tx, ty) in directions:
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        trans_img = cv2.warpAffine(image, M, (width, height))
        trans_img = np.expand_dims(trans_img,2)
        
        if initial:
            translated_images = trans_img
            initial=False
        else:
            translated_images = np.concatenate([translated_images,trans_img],-1)
    
    return translated_images

In [ ]:
name_count = 0
for category in categories:
    image_path = "./"+category+"/val/image/"
    json_path = "./"+category+"/val/coco_format-mask_encoding=rle-instances_2017.json"
    
    with open(json_path,'r') as file:
        json_data = json.load(file)

    data_dict = dict()
    distances = [1,2,4,8,16,32,64]

    for idx,annotation_data in enumerate(json_data['annotations']):  # type(annotation_data) = dict
        key = int(annotation_data['image_id'])
        if key not in data_dict:
            data_dict[key] = set()
        data_dict[key].add(idx)

    for image_id in data_dict:
        # 데이터 매치 문제없는지 확인
        for idx in data_dict[image_id]:
            assert image_id == json_data['annotations'][idx]['image_id'], "key:value idx match error"
        # image id랑 key랑 같은지 확인
        assert image_id == json_data['images'][image_id-1]['id'], "img index:key match error"

        image_name = json_data['images'][image_id-1]['file_name']
        #image = cv2.imread(image_path+image_name)

        #--------intance label create--------
        # 객체 id 초기화
        instance_id = 1
        # 라벨 데이터 틀 생성
        instance_label = np.zeros((1024,1024))
        for i in data_dict[image_id]:
            binary_mask = decode_mask.decode(json_data['annotations'][i]['segmentation'])  # 1 or 0
            binary_mask=np.where(binary_mask==1,instance_id,0)
            instance_label += binary_mask
            instance_id+=1
        #--------intance label complete--------

        #--------affinity label create--------
        initial = True
        for d in distances:  # 총 7개의 거리
            img_8 = translate_image_8_directions(instance_label,d)  # d 거리에 대해 8방향으로 이동한 이미지 생성
            if initial:
                affinity_label = img_8
                initial = False
            else:
                affinity_label = np.concatenate([affinity_label,img_8],-1)

        ins_label_56 = np.repeat(np.expand_dims(instance_label,-1), 56, axis=2)
        affinity_label = np.where((affinity_label == ins_label_56)&(ins_label_56>0), 1, 0)  # instance label과 객체 id가 같으면 1 다르면 0으로
        #--------affinity label complete--------

        # 이미지 및 affinity 라벨 저장
        src_path = image_path+image_name
        dst_path = "./_affinity/val/image/"+str(name_count)+".jpg"
        copyfile(src_path,dst_path)

        dst_path = "./_affinity/val/label/"+str(name_count)+".npz"
        np.savez_compressed(dst_path,data=affinity_label)

        name_count+=1

In [ ]:
categories = ['antenna']
for category in categories:
    image_path = "./"+category+"/val/image/"
    json_path = "./"+category+"/val/coco_format-mask_encoding=rle-instances_2017.json"
    
    with open(json_path,'r') as file:
        json_data = json.load(file)

    data_dict = dict()
    distances = [1,2,4,8,16,32,64]

    for idx,annotation_data in enumerate(json_data['annotations']):  # type(annotation_data) = dict
        key = int(annotation_data['image_id'])
        if key not in data_dict:
            data_dict[key] = set()
        data_dict[key].add(idx)

    for image_id in data_dict:

        # 데이터 매치 문제없는지 확인
        for idx in data_dict[image_id]:
            assert image_id == json_data['annotations'][idx]['image_id'], "key:value idx match error"
        # image id랑 key랑 같은지 확인
        assert image_id == json_data['images'][image_id-1]['id'], "img index:key match error"

        image_name = json_data['images'][image_id-1]['file_name']
        #image = cv2.imread(image_path+image_name)

        #--------intance label create--------
        # 객체 id 초기화
        instance_id = 1
        # 라벨 데이터 틀 생성
        instance_label = np.zeros((1024,1024))
        for i in data_dict[key]:
            binary_mask = decode_mask.decode(json_data['annotations'][i]['segmentation'])  # 1 or 0
            binary_mask=np.where(binary_mask==1,instance_id,0)
            instance_label += binary_mask
            instance_id+=1
        #--------intance label complete--------

        #--------affinity label create--------
        initial = True
        for d in distances:  # 총 7개의 거리
            img_8 = translate_image_8_directions(instance_label,d)  # d 거리에 대해 8방향으로 이동한 이미지 생성
            if initial:
                affinity_label = img_8
                initial = False
            else:
                affinity_label = np.concatenate([affinity_label,img_8],-1)

        ins_label_56 = np.repeat(np.expand_dims(instance_label,-1), 56, axis=2)
        affinity_label = np.where((affinity_label == ins_label_56)&(ins_label_56>0), 1, 0)  # instance label과 객체 id가 같으면 1 다르면 0으로
        #--------affinity label complete--------
        
        
        # 이미지 및 affinity 라벨 저장
        src_path = image_path+image_name
        dst_path = "./_affinity/val/image/"+str(name_count)+".jpg"
        copyfile(src_path,dst_path)

        dst_path = "./_affinity/val/label/"+str(name_count)+".npz"
        np.savez_compressed(dst_path,data=affinity_label)

        name_count+=1

# semantic label

In [1]:
from shutil import copyfile

In [2]:
import os,cv2
import numpy as np

In [1]:
key_dict = {
'fence':1,
'branch':2,
'log':3,
'hanger':4,
'wire':5,
'antenna':6
} 

In [4]:
categories = ['fence','branch','log','hanger','wire',]

In [5]:
categories

['fence', 'branch', 'log', 'hanger', 'wire']

In [6]:
# move img files

name_count = 0
for category in categories:
    file_names = os.listdir("./"+category+"/train/image/")
    
    try:
        file_names.remove('.ipynb_checkpoints')
    except:
        pass
    
    for name in file_names:
        src_path = "./"+category+"/train/image/"+name
        dst_path = "./_semantic/image/"+str(name_count)+".jpg"
        copyfile(src_path,dst_path)
        
        name = name.split(".")[0]
        src_path = "./"+category+"/train/instance_map/"+name+".png"
        dst_path = "./_semantic/label/"+str(name_count)+".png"
        label_img = cv2.imread(src_path)
        label_img = np.where(label_img>=1,key_dict[category],0)
        cv2.imwrite(dst_path,label_img)
        
        name_count+=1

In [7]:
categories = ['antenna']

for category in categories:
    file_names = os.listdir("./"+category+"/train/img/image/")
    
    try:
        file_names.remove('.ipynb_checkpoints')
    except:
        pass
    
    for name in file_names:
        src_path = "./"+category+"/train/img/image/"+name
        dst_path = "./_semantic/image/"+str(name_count)+".jpg"
        copyfile(src_path,dst_path)
        
        name = name.split(".")[0]
        src_path = "./"+category+"/train/instance/instance_map/"+name+".png"
        dst_path = "./_semantic/label/"+str(name_count)+".png"
        label_img = cv2.imread(src_path)
        label_img = np.where(label_img>=1,key_dict[category],0)
        cv2.imwrite(dst_path,label_img)
        
        name_count+=1

## valid data

In [4]:
categories = ['fence','branch','log','hanger','wire','antenna']

In [5]:
name_count = 0
for category in categories:
    file_names = os.listdir("./"+category+"/val/image/")
    
    try:
        file_names.remove('.ipynb_checkpoints')
    except:
        pass
    
    for name in file_names:
        src_path = "./"+category+"/val/image/"+name
        dst_path = "./_semantic/val/image/"+str(name_count)+".jpg"
        copyfile(src_path,dst_path)
        
        name = name.split(".")[0]
        src_path = "./"+category+"/val/instance_map/"+name+".png"
        dst_path = "./_semantic/val/label/"+str(name_count)+".png"
        label_img = cv2.imread(src_path)
        label_img = np.where(label_img>=1,key_dict[category],0)
        cv2.imwrite(dst_path,label_img)
        
        name_count+=1

In [6]:
len(os.listdir("./_semantic/val/image/"))

2685

In [8]:
len(os.listdir("./_semantic/val/label/"))

2685